# About this code
This is an enumerator for the connected partitions of grid graphs. Where we ask how many ways there are to cut an $m\times n$ grid into $k$ connected pieces, where we can consider both *rook* connectivity, meaning that two squares are considered adjacent if and only if they meet at an edge, and *queen* connectivity where we also consider two squares which meet at a corner to be adjacent.

There are a very large number of such partitions.  For the $7\times 7$ grid, there are almost 150 million ways to partition it into seven rook-connected pieces of size seven.  However, for smaller instances of the problem, we can count the number of such partitions relatively quickly, and that is what this code does.  We include the option to actually output the list of partitions as well.

This notebook walks through the algorithm and its proof-of-correctness.  The docstrings in the methods give a sketch of the functionality, and more details can be found in the written text.  A command line version of this tool is coming soon.

### The Julia Language
This code is written in Julia, a relatively new language designed by some cool folks at MIT for scientific and numerical applications.  Julia blends the usability of Python with the (almost) speed of C with a MATLAB-like syntax.  It is just-in-time compiled, so it's not unusual for the first execution of a method (what functions and stuff are called in Julia) to be slow, since the compiler needs to do its thing, but subsequent calls are fast.  As a relative benchmark, this exact same algorithm in Python runs about ten times slower.

Julia is functional rather than object-oriented like Python, and it's typing, polymorphic methods, and homoiconism are really cool and let you do some neat things, and I definitely recommend checking out more of the language's features if you're interested.

In [1]:
# These are the packages we need to use.  If you don't
# have one, uncomment and run in the next block the
# appropriate lines and then run this block again

using Pkg
using LightGraphs
using IterTools
using DataStructures
using Base
using Combinatorics

In [2]:
#Pkg.add("LightGraphs")
#Pkg.add("IterTools")
#Pkg.add("DataStructures")
#Pkg.add("Base")
#Pkg.add("Combinatorics")

### Ominos
The first fundamental observation to make is that any valid partitioning of the grid is uniquely specified by the collection of pieces (which we call *ominos*) which make it up.  Therefore, instead of starting with the whole grid and trying to cut it up, we'll instead start with all of the pieces and figure out which are the valid ways to put those pieces together.

Given a set of ominos, a *valid $k$-partition of the $m\times n$ grid* is a collection of $k$ ominos from that set such that none overlap and they together exactly cover the grid.

We begin with several methods to construct these sets of ominos.

In [3]:
"""
Checks if an omino is 'valid'. 
A valid omino is a binary matrix where
all of the ones are positioned such that
they form a single (rook/queen)-connected
component.
"""
function valid_omino(omino,cont="rc")
    if 2 in omino
        return false
    end
    n1 = size(omino)[1]
    n2 = size(omino)[2]
    check = zeros(Int8, n1,n2)
    _f = false
    g = Grid([n1,n2])
 
    if cont == "qc" 
        for i=1:n1-1
           for j=1:n2-1
               add_edge!(g,((i-1)*n2+j),i*n2+j+1) 
            end
        end
        
        for i=1:n1-1
            for j=2:n2
                add_edge!(g,((i-1)*n2+j),i*n2+j-1)
            end
        end
    end
    
    for i=n1:-1:1
        for j=n2:-1:1
           if omino[i,j] ==0
                rem_vertex!(g,((i-1)*n2+j))
            end  
        end   
    end
    
    return length(connected_components(g)) == 1  
end
    



"""
Given a list of indices [(i_1,j_1),(i_2,j_2),...]
checks if setting the values at those indices in
a grid of size grid[1]xgrid[2] yields a valid
omino as according to the specified continuity
rule.  

If so, returns that omino.  If not,
returns the grid[1]xgrid[2] zeros matrix.
"""
function make_omino(inds,grid,cont="rc")
    
    p = zeros(Int8,grid[1],grid[2])
    for loc in inds
        p[loc[1],loc[2]]=1
    end
    
    if valid_omino(p,cont)
        return p
    else
        return zeros(Int8,grid[1],grid[2])
    end
end


"""
Returns the set of valid grid[1]xgrid[2] ominos, 
according to the continuity rule, which have 
cells ones.

Note: starting with make_omino_set(1,[...]) and
calling biggen(...) in a loop is generally much
faster than this method.  Since this operates
by combinatorial enumeration, checking all
possible subsets of indices.
"""
function make_omino_set(cells,grid,cont="rc")
    
   pair_idx = []
    for i=1:grid[1]
        for j=1:grid[2]
            push!(pair_idx, (i,j))
        end
    end
    
    ominos = []
    for t in subsets(pair_idx, Val{cells}())
        p = make_omino(t,grid,cont)
        
        if sum(p)!=0
            push!(ominos,p)
        end
    end   
    return ominos    
end

make_omino_set

biggen_graph (generic function with 1 method)

### Building omino sets
The correctness of `make_omino_set()` is somewhat obvious, since it checks all possible binary matrices with `cells` ones for validity. However, this method is also somewhat obviously slow.  By making the observation that any $(k+1)$-omino can be thought of by adding one cell to a $k$-omino, we can dramatically decrease the time needed to construct the omino sets.  This is what the following method, `biggen()` does -- it takes a set of ominos and returns the set of ominos reachable by augmenting something in that set.

If we start with the set of $1$-ominos on a grid (the set of $m\times n$ binary matrices with a single one and call `biggen()` recursively $k-1$ times, we can construct every $k$-omino.  The proof is by induction, and follows immediately from the above observation.  Since every $(k+1)$-omino is an augmented $k$-omino, if we know we have the complete set of $k$-ominos, then `biggen()` will yield the complete set of $(k+1)$-ominos.  Since we start with the complete set of $1$-ominos by assertion, the base case holds as well.



In [4]:
"""
Takes as input a collection of ominos
and returns the list of valid ominos on
the same shape grid which can be formed
by changing a single zero to a one in
any element of omlist.
"""
function biggen(omlist,cont="rc")
    grid = size(omlist[1])
    new_oms = Set()
    for o in omlist
        
        for i = 1:grid[1]
            for j = 1:grid[2]
                z = zeros(Int8,grid[1],grid[2])
                z[i,j]+=1
                if o[i,j]==0 && valid_omino(z+o,cont)
                    push!(new_oms,z+o)                 
                end
            end
        end       
    end
    return collect(new_oms)
end

biggen

# Checking for bad holes

Since we're trying to enumerate a collection of exponentially many things, we need to use all of the tricks we can find to pare down the space of objects we have to evaluate.  One such trick is to discard ominos that are incompatible with *any* partitioning, since they have 'holes' which cannot be filled by any other ominos.  To illustrate, consider the omino below:

 

    0 1 0
    1 1 0
    0 0 0
 

If we're looking at partitions of the $3\times 3$ grid into three pieces of size three, then this omino can't be part of any valid partition, since there's no other omino compatible with this which will let us fill in the top left corner.  As such, we should throw this one away.

This observation yield the following fact:

An omino is incompatible with any partition into pieces of size $p_1,p_2,\dots,p_\ell$ if it has a hole whose size cannot be written as the sum of $p_i$ in any way.

We'll make the additional observation that the set of sizes of holes which cannot be filled at least includes everything smaller than the minimum $p_i$ as well as anything in between the maximum $p_1$ and twice the minimum $p_i$.  We note as well that this set may be empty, as in the case where we allow the use of $1$-ominos.

In [5]:
"""
Given an omino, and a collection of bad
hole sizes (bad_holes), returns true
if none of the connected components formed
by the ominos zeros are of a size in that
list.

For example if we're looking at splitting the 3x3
grid into 3 rook connected pieces of size 3, then we
might have bad_holes=[1,2,4,5], since it's going to
be impossible to add any collection of our 3-ominos
to this omino to complete the partitioning.  Therefore,
if we call this method on

0 1 0
1 1 0
0 0 0

it would return false.
"""
function check_holes(omino, bad_holes,cont="rc")

    n1 = size(omino)[1]
    n2 = size(omino)[2]
    check = ones(Int8, n1,n2)-omino
    _f = false
    g = Grid([n1,n2])
     
    if cont == "qc"
        for i=1:n1-1
           for j=1:n2-1
               add_edge!(g,((i-1)*n2+j),i*n2+j+1  ) 
            end
        end
        
        for i=1:n1-1
            for j=2:n2
                add_edge!(g,((i-1)*n2+j),i*n2+j-1)
            end
        end        
    end
    
    for i=n1:-1:1
        for j=n2:-1:1
           if check[i,j] ==0
                rem_vertex!(g,((i-1)*n2+j))
            end            
        end   
    end
    
    for c in connected_components(g)
        if length(c) in bad_holes
            return false
        end
    end
    
    return true
end


check_holes

### Defining the tree structure

We now make the next observation, that given any partitioning into ominos, we can canonically order those ominos by reading them off of the grid left-to-right, top-to-bottom.  Each partition has a unique canonical order, which is going to prevent us from double-counting any partitions as we go along.  

Because of this ordering, we can talk about *partial partitions* in a meaningful way.  We define a partial partition to be the truncation of a complete partition.  For example, if $o_1,o_2,o_3,o_4$ is a full partition into four pieces, then $o_1,o_2$ is its (unique) length-two partial partition.  By the construction of the canonical ordering, we can also observe that if $o_i$ and $o_j$ are two ominos which appear together in some plan, then either $o_i$ *always* precedes $o_j$ or $o_j$ *always* precedes $o_i$ in any plan.  This is because if we have some plan where $o_i$ precedes $o_j$, then it must be the case that some cell of $o_i$ occurs before some cell of $o_j$ in the left-to-right, top-to-bottom ordering of the grid, so in every canonical ordering of any partition including both $o_i$ and $o_j$, $o_i$ comes first.

This uniqueness lets us imagine a tree structure on the set of valid partitions.  The root is the empty list, which is the unique length-zero partial partition and is the length-zero partial partition for every partition.  The leaves of the tree are the valid partitions which we are trying to enumerate.  The level-$i$ nodes correspond to length-$i$ partial partitions, and since the canonical ordering of the chunks of a partition induce a well-defined partial order on the ominos, this tree structure is well-defined.

Our next trick to pare down the search space is to build a dictionary of conflicts which tells us which ominos are compatible with which other ominos.  If there are two ominos which overlap, it's a huge waste of time to try to explore partial plans which contain both of them, so this dictionary helps us avoid that.  We'll say that two ominos are compatible if they do not overlap and their combination doesn't result in a partial plan which has unfillable holes.

While we're here, we're also going to build a dictionary which, for each entry of the grid, tells us the collection of ominos which have their first one in that entry.  This again lets us pare down the search space, since we don't want to waste time checking partial plans which have gaps between the $k$th and $k+1$th pieces.


In [6]:
"""
Given a list of ominos, a list of bad hole sizes, the number
of parts we want to partition into, and a continuity rule,
generates two dictionaries to guide the dynamic program.

It is important that omlist indeed be a list, since we do our
hashing by indexing into that list, so it's order needs
to be fixed.

The first dictionary, firstdict, has a key for each (row,col)
pair to index into the ominos.  The values are the set of ominos
for which first_one(omino)==(row,col).  This will allow the 
algorithm to easily ask for the list of ominos which have their
first one in a particular index.

The second dictionary, conflict, has a key for each omino and
the values are the set of ominos which are 'compatible' with
the key.  Here, compatibility means they do not overlap, and
combining both of them does not create any bad holes.  These
are necessary conditions for the pair of ominos to be able
to be used together in some partitioning.
"""
function build_conflicts(omlist, bad_holes,cont="rc")
    
    
    firstdict = DefaultDict(Set)
    
    for o in 1:length(omlist)
        k = first_one(omlist[o])
        push!(firstdict[k],o)
    end
    
    conflict = DefaultDict(Set)
    for os in subsets(1:length(omlist),Val{2}())
        i = os[1]
        j = os[2]
        _c = true
        for p in zip(omlist[i],omlist[j])
            if _c && p[1]*p[2] ==1
                _c = false
            end
        end
        if _c && check_holes(omlist[i]+omlist[j],bad_holes,cont)
            push!(conflict[i],j)
            push!(conflict[j],i)
        end        
    end
    return firstdict,conflict
end

build_conflicts

### How to navigate the tree

We need one last piece before we can be satisfied with the algorithm, and that is to know how to navigate this tree structure.  Since we read off the ominos from left-to-right, top-to-bottom, we know that any partial partition of length 1 must fill the upper-left corner of the grid.  From there, we can proceed by induction to observe that if we have a length-$k$ partial partition, then any valid partition which includes it must be such that the $k+1$th omino fills the first entry in the grid left empty by the first $k$ pieces.

The next two functions allow us to grab the first zero from a partial plan to match with the first one in a candidate $k+1$th omino.

In [7]:
"""
Returns the first (lexicographically)
(row,column) index where the entry
of mat is zero.
"""
function first_zero(mat)
    
   for i=1:size(mat,1)
        for j=1:size(mat,2)
            if mat[i,j]==0
                return (i,j)
            end
        end
    end
    return nothing  
end

"""
Returns the first (lexicographically)
(row,column) index where the entry
of mat is one.
"""
function first_one(mat)
    
   for i=1:size(mat,1)
        for j=1:size(mat,2)
            if mat[i,j]==1
                return (i,j)
            end
        end
    end
    return nothing 
end

first_one

### The enumerator

Thus, the algorithm proceeeds by recursive depth-first search of the tree, counting all of the leaves it finds.  It begins by starting with the length-one partial partitions, being the set of ominos which fill the upper-left corner of the grid.  For each of these, it recursively finds the length-$k+1$ partial partitions by trying to add each omino which fills the first space left empty by the first $k$ ominos.

In [8]:
"""
The enumerator. Takes five arguments:
- grid: a pair [m,n] which give the dimension of
        the grid to be partitioned
- om_sizes: a list of integers [a,b,...] which 
            are the allowed sizes of ominos
- num_parts: an integer, the number of parts
- cont: a string "rc" or "qc" to give a continuity rule
- io: a boolean, whether or not to write the list of partitions
      to a file.

returns: an integer, counting the number of valid partitions

usage: to partition the 4x6 grid into 4 rook connected pieces
       of size 4,5,6, with file output, you would call
       enumerator([4,6],[4,5,6],4,"rc",true)

The output format is a text file, where each partition
is on its own line.  The components are numbered 1,2,...num_parts
and the linear order corresponds with the lexicographic order of the
indexing of the grid.  For example, the representation of

1 2 2
1 1 2
3 3 3

is

1,2,2,1,1,2,3,3,3

The file naming convention is [<m>,<n>]_[<p_1>,<p_2>,...,<p_l>]_k_<cont>.txt where 
m and n are the dimensions of the grid, the p_i are the allowed sizes of the chunks, 
k is the number of pieces, and cont is either rc for rook contiguity or qc for queen contiguity.



includes three helper methods defined locally:

verify(a,b) checks if the ominos at indices a,b
conflict with each other as according to the 
conflict dictionary constructed by make_conflicts()

get_next() takes a partial partitioning and gives the
collection of ominos which have their first one where
the partial partition has its first zero and which do not
conflict with any of the ominos in the partial partition

recurs_part() uses get_next() to recursively build
all of the partitions.  If it finds a valid one,
it increments the counter and (if io==true) writes
it to the output file
"""
function enumerator(grid, om_sizes,num_parts, cont="rc", io=false)
    count = 0
    oms = []
    bad_hole_sizes = [ [i for i = 1:minimum(om_sizes)-1] ;[i for i = maximum(om_sizes)+1:2*minimum(om_sizes)-1]        ]
    
    tmp_oms = make_omino_set(1,grid,cont)
    if io
        os = om_sizes
        gr = grid
        np = num_parts
        outfile = open(replace("enumerations/enum_$(gr)_$(os)_$(np)_$(cont).txt"," "=>""),"w")
    end
    
    
    if 1 in om_sizes
        oms = [oms;tmp_oms]
    end
    
    for sz = 2:maximum(om_sizes)
        tmp_oms = biggen(tmp_oms,cont)
        if sz in om_sizes
            oms = [oms;tmp_oms]
        end
        #print("MADE SZ ",sz ,' ',length(tmp_oms),"\n")


    end
    
    
    oms = [ o for o in oms if check_holes(o,bad_hole_sizes,cont)]

    #print("MADE OMINOS\n")
    
    
    
    fd,cf = build_conflicts(oms,bad_hole_sizes,cont)
    #print("MADE DICTIONARIES\n")           
    
    function verify(plan)
        #print(plan,'\n')
        
        for p in subsets(plan,Val{2}())
            if !(p[2] in cf[p[1]])
                return false
            end
        end
        return true
    end
           

    function get_next(fd, curr_inds, omlist) 
       return [d for d in fd[first_zero(sum([omlist[j] for j in curr_inds]))] if verify(curr_inds + d)]     
    end                  
                
    function recurs_part(plan,steps)
        #print("plan:",plan,"  sum: ",sum([oms[j] for j in plan]),'\n')
        if steps == 0
            if sum(sum( [oms[j] for j in plan])) == grid[1]*grid[2] #&& sort!(collect([sum(oms[j]) for j in plan    ] )) == om_sizes
                count +=1
                if io
                    strrep = "$(sum([j.*oms[plan[j]] for j in 1:length(plan)]))\n"
                    strrep = replace(strrep," "=>",")
                    strrep = replace(strrep,"["=>"")
                    strrep = replace(strrep,";"=>"")
                    strrep = replace(strrep,"]"=>"")
                    write(outfile,  strrep)
                end
                return
            end
        elseif sum(sum( [oms[j] for j in plan])) == grid[1]*grid[2]
        return
        end
        for q in fd[first_zero( sum([oms[j] for j in plan     ]))]
            if verify([plan;[q]])
                recurs_part([plan;[q]],steps-1)
            end
        end
    end
    
    #print("NEED TO DO ",length(fd[(1,1)]), " TOP-LEVELS\n")
    for p in fd[(1,1)]
        #print(" ",p," ")
        recurs_part([p],num_parts-1)
    end
    
    
   print('\n')
    if io
        close(outfile)
    end
   return count 
    
end

enumerator

### Running the code

We can run the enumerator like this.  I like to use the `@time` macro, which outputs the time it takes for the following function to run.  

This is how you would find the rook contiguous partitions of the $4\times 6$ grid into 4 pieces of size 5, 6, or 7.

In [17]:
#@time print(enumerator([4,6],[6],4,"rc",false))


2369  0.555473 seconds (6.43 M allocations: 489.308 MiB, 11.69% gc time)


# BELOW HERE IS WORK-IN-PROGRESS

In [220]:
function valid_graph_om(omino,graph)
    if length(omino)!=nv(graph)
        print("BAD INPUT\n")
        return
    end
    g = induced_subgraph(graph, find(omino .== 1))
    #=
    for i=nv(graph):-1:1
        #print("HERE $i\n")
       if omino[i]==0
            rem_vertex!(g,i)
        
        end    
    end
    =#
    #print("ENDED LOOP! $(collect(connected_components(g))) \n")
    return length(collect(connected_components(g)))==1

end




function biggen_graph(omlist, graph)
    if length(omlist[1])!=nv(graph)
        print("BAD INPUT\n")
        return
    end
    newoms = Set()
    for omino in omlist
        
        for i=1:length(omino)
            z = zeros(Int8,size(omino))[i]+=1
                if valid_graph_om(omino+z,graph)
                    push!(newoms,omino)
                end
            
        end
    end
    return collect(newoms)
end




biggen_graph (generic function with 1 method)

In [182]:
function check_graph_holes(om,graph,bad_holes)
    if length(om)!=nv(graph)
        print("BAD INPUT\n")
        return
    end
    omino = ones(Int8,size(om))-om
    g = deepcopy(graph)
    for i=nv(graph):-1:1
        #print("HERE $i\n")
       if omino[i]==0
            rem_vertex!(g,i)
        
        end    
    end
    #print("ENDED LOOP! $(collect(connected_components(g))) \n")
    for c in collect(connected_components(g))
        #print("$(length(c))\n")
        if length(c) in bad_holes
            return false
        end
    end
    return true
end

check_graph_holes (generic function with 1 method)

In [183]:
gr = make_iowa_graph()
print(gr)

{99, 219} undirected simple Int64 graph

In [179]:
n1 = []
for i=1:99
    s = zeros(Int8,1,99)
    s[i]=1
    push!(n1,s)
end
print(length(n1[1]))
print(nv(gr))

9999

In [180]:
n2 = biggen_graph(n1,gr)
n2g = [ o for o in n2g if check_graph_holes(o,gr,[97])]
print(length(n2g))
print(n2g[1])

97
97
0

BoundsError: BoundsError: attempt to access 0-element Array{Array{Int8,2},1} at index [1]

In [201]:
dists = []
for i=1:99
    s = zeros(Int8,1,99)
    s[i]=1
    push!(dists,s)
end
print(length(dists),"\n")

for i=1
    dists = [ d for d in dists if check_graph_holes(d,gr,[1])]
    dists = biggen_graph(dists,gr) 
    
end
print(length(dists),"\n")
print(dists[2])

99
219
Int8[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

In [204]:
function build_graph_conflicts(omlist, gr,bad_holes)
    
    
    firstdict = DefaultDict(Set)
    
    for o in 1:length(omlist)
        k = first_one(omlist[o])
        push!(firstdict[k],o)
    end
    
    conflict = DefaultDict(Set)
    for os in subsets(1:length(omlist),Val{2}())
        i = os[1]
        j = os[2]
        _c = true
        for p in zip(omlist[i],omlist[j])
            if _c && p[1]*p[2] ==1
                _c = false
            end
        end
        if _c && check_graph_holes(omlist[i]+omlist[j],gr,bad_holes)
            push!(conflict[i],j)
            push!(conflict[j],i)
        end        
    end
    return firstdict,conflict
end

build_graph_conflicts (generic function with 1 method)

In [202]:
fd,cf = build_graph_conflicts(dists,gr,[1])

(DefaultDict{Any,Any,UnionAll}((1, 47)=>Set(Any[148, 161, 65]),(1, 23)=>Set(Any[37, 2, 176]),(1, 26)=>Set(Any[121, 33]),(1, 76)=>Set(Any[174, 137, 5]),(1, 15)=>Set(Any[34]),(1, 35)=>Set(Any[206, 32, 104]),(1, 36)=>Set(Any[132, 179, 205]),(1, 25)=>Set(Any[213, 67, 96]),(1, 10)=>Set(Any[13, 177, 3]),(1, 12)=>Set(Any[87, 73])…), DefaultDict{Any,Any,UnionAll}(11=>Set(Any[134, 158, 160, 215, 131, 207, 173, 74, 201, 176  …  6, 24, 73, 119, 156, 53, 116, 23, 27, 218, 56]),134=>Set(Any[11, 158, 215, 160, 29, 131, 207, 173, 74, 201  …  6, 24, 73, 119, 156, 53, 116, 23, 27, 218, 56]),158=>Set(Any[11, 134, 215, 160, 29, 131, 207, 173, 74, 201  …  6, 24, 73, 119, 156, 53, 116, 23, 27, 218, 56]),160=>Set(Any[11, 134, 158, 215, 29, 131, 207, 173, 74, 201  …  6, 24, 73, 119, 156, 53, 116, 23, 27, 218, 56]),215=>Set(Any[11, 134, 158, 160, 29, 131, 207, 173, 74, 201  …  22, 6, 73, 119, 156, 53, 116, 23, 27, 218, 56]),29=>Set(Any[134, 158, 160, 215, 131, 207, 173, 74, 201, 176  …  6, 24, 73, 119, 156, 5

In [217]:
function iowa_enumerator(om_sizes,num_parts, io=false)
    count = 0
    oms = []
    bad_hole_sizes = []#[ [i for i = 1:minimum(om_sizes)-1] ;[i for i = maximum(om_sizes)+1:2*minimum(om_sizes)-1]        ]
    iagr = make_iowa_graph()
    tmp_oms = []
    for i=1:99
        s = zeros(Int8,1,99)
        s[i]=1
        push!(tmp_oms,s)
    end
    if io
        os = om_sizes
        
        np = num_parts
        outfile = open(replace("enumerations/iowaenum_$(os)_$(np)_$(cont).txt"," "=>""),"w")
    end
    
    
    if 1 in om_sizes
        oms = [oms;tmp_oms]
    end
    
    for sz = 2:maximum(om_sizes)
        tmp_oms = biggen_graph(tmp_oms,iagr)
        if sz in om_sizes
            oms = [oms;tmp_oms]
        end
        print("MADE SZ ",sz ,' ',length(tmp_oms),"\n")


    end
    
    
    oms = [ o for o in oms if check_graph_holes(o,iagr,bad_hole_sizes)]

    print("MADE OMINOS\n")
    
    
    
    fd,cf = build_graph_conflicts(oms,iagr,bad_hole_sizes)
    print("MADE DICTIONARIES\n")           
    
    function verify(plan)
        #print(plan,'\n')
        
        for p in subsets(plan,Val{2}())
            if !(p[2] in cf[p[1]])
                return false
            end
        end
        return true
    end
           

    function get_next(fd, curr_inds, omlist) 
       return [d for d in fd[first_zero(sum([omlist[j] for j in curr_inds]))] if verify(curr_inds + d)]     
    end                  
                
    function recurs_part(plan,steps)
        #print("plan:",plan,"  sum: ",sum([oms[j] for j in plan]),'\n')
        if steps == 0
            if sum(sum( [oms[j] for j in plan])) == nv(iagr) #&& sort!(collect([sum(oms[j]) for j in plan    ] )) == om_sizes
                count +=1
                if io
                    strrep = "$(sum([j.*oms[plan[j]] for j in 1:length(plan)]))\n"
                    strrep = replace(strrep," "=>",")
                    strrep = replace(strrep,"["=>"")
                    strrep = replace(strrep,";"=>"")
                    strrep = replace(strrep,"]"=>"")
                    write(outfile,  strrep)
                end
                return
            end
        elseif sum(sum( [oms[j] for j in plan])) == nv(iagr)
        return
        end
        for q in fd[first_zero( sum([oms[j] for j in plan     ]))]
            if verify([plan;[q]])
                recurs_part([plan;[q]],steps-1)
            end
        end
    end
    
    print("NEED TO DO ",length(fd[(1,1)]), " TOP-LEVELS\n")
    for p in fd[(1,1)]
        print(" ",p," ")
        recurs_part([p],num_parts-1)
    end
    
    
   print('\n')
    if io
        close(outfile)
    end
   return count 
    
end

iowa_enumerator (generic function with 3 methods)

MADE SZ 2 219
MADE OMINOS
MADE DICTIONARIES
NEED TO DO 3 TOP-LEVELS
 162 

InterruptException: InterruptException:

In [177]:
function make_iowa_graph()
    
    g = SimpleGraph(99)
    
add_edge!(g,1,2)
add_edge!(g,1,12)
add_edge!(g,2,3)
add_edge!(g,2,13)
add_edge!(g,3,4)
add_edge!(g,3,14)
add_edge!(g,4,5)
add_edge!(g,4,15)
add_edge!(g,5,6)
add_edge!(g,5,15)
add_edge!(g,5,16)
add_edge!(g,6,16)
add_edge!(g,6,7)
add_edge!(g,7,8)
add_edge!(g,7,17)
add_edge!(g,8,9)
add_edge!(g,8,17)
add_edge!(g,8,18)
add_edge!(g,9,10)
add_edge!(g,9,19)
add_edge!(g,10,11)
add_edge!(g,10,19)
add_edge!(g,10,20)
add_edge!(g,11,21)
add_edge!(g,12,13)
add_edge!(g,12,22)
add_edge!(g,13,23)
add_edge!(g,13,14)
add_edge!(g,14,15)
add_edge!(g,14,24)
add_edge!(g,15,25)
add_edge!(g,16,17)
add_edge!(g,16,27)
add_edge!(g,17,18)
add_edge!(g,17,28)
add_edge!(g,18,19)
add_edge!(g,18,29)
add_edge!(g,19,20)
add_edge!(g,19,30)
add_edge!(g,20,21)
add_edge!(g,20,30)
add_edge!(g,20,40)
add_edge!(g,21,41)
add_edge!(g,21,42)
add_edge!(g,22,23)
add_edge!(g,22,31)
add_edge!(g,23,24)
add_edge!(g,23,31)
add_edge!(g,23,32)
add_edge!(g,24,25)
add_edge!(g,24,33)
add_edge!(g,25,26)
add_edge!(g,25,34)
add_edge!(g,25,35)
add_edge!(g,26,27)
add_edge!(g,26,35)
add_edge!(g,27,28)
add_edge!(g,27,35)
add_edge!(g,27,36)
add_edge!(g,28,29)
add_edge!(g,28,37)
add_edge!(g,29,30)
add_edge!(g,29,38)
add_edge!(g,29,39)
add_edge!(g,30,39)
add_edge!(g,31,32)
add_edge!(g,31,43)
add_edge!(g,32,33)
add_edge!(g,32,44)
add_edge!(g,33,34)
add_edge!(g,33,44)
add_edge!(g,33,45)
add_edge!(g,34,35)
add_edge!(g,34,45)
add_edge!(g,34,46)
add_edge!(g,35,36)
add_edge!(g,35,46)
add_edge!(g,35,47)
add_edge!(g,36,37)
add_edge!(g,36,47)
add_edge!(g,36,48)
add_edge!(g,37,38)
add_edge!(g,37,48)
add_edge!(g,37,49)
add_edge!(g,38,39)
add_edge!(g,38,49)
add_edge!(g,38,50)
add_edge!(g,39,40)
add_edge!(g,39,50)
add_edge!(g,39,51)
add_edge!(g,40,41)
add_edge!(g,40,51)
add_edge!(g,40,52)
add_edge!(g,41,42)
add_edge!(g,41,52)
add_edge!(g,41,53)
add_edge!(g,42,53)
add_edge!(g,42,54)
add_edge!(g,43,44)
add_edge!(g,43,55)
add_edge!(g,44,45)
add_edge!(g,44,55)
add_edge!(g,44,56)
add_edge!(g,45,46)
add_edge!(g,45,57)
add_edge!(g,45,58)
add_edge!(g,46,47)
add_edge!(g,46,58)
add_edge!(g,46,58)
add_edge!(g,47,48)
add_edge!(g,47,59)
add_edge!(g,47,60)
add_edge!(g,48,49)
add_edge!(g,48,60)
add_edge!(g,48,61)
add_edge!(g,49,50)
add_edge!(g,49,61)
add_edge!(g,50,51)
add_edge!(g,50,62)
add_edge!(g,51,52)
add_edge!(g,51,63)
add_edge!(g,52,53)
add_edge!(g,52,64)
add_edge!(g,52,65)
add_edge!(g,53,54)
add_edge!(g,53,65)
add_edge!(g,53,66)
add_edge!(g,54,66)
add_edge!(g,55,56)
add_edge!(g,55,69)
add_edge!(g,56,57)
add_edge!(g,56,69)
add_edge!(g,56,70)
add_edge!(g,57,58)
add_edge!(g,57,70)
add_edge!(g,58,59)
add_edge!(g,58,71)
add_edge!(g,59,60)
add_edge!(g,59,72)
add_edge!(g,60,61)
add_edge!(g,60,73)
add_edge!(g,60,74)
add_edge!(g,61,62)
add_edge!(g,71,74)
add_edge!(g,61,75)
add_edge!(g,62,63)
add_edge!(g,62,75)
add_edge!(g,62,76)
add_edge!(g,63,64)
add_edge!(g,63,76)
add_edge!(g,63,77)
add_edge!(g,64,65)
add_edge!(g,64,67)
add_edge!(g,64,77)
add_edge!(g,64,78)
add_edge!(g,65,66)
add_edge!(g,65,67)
add_edge!(g,65,68)
add_edge!(g,66,68)
add_edge!(g,67,78)
add_edge!(g,67,68)
add_edge!(g,69,70)
add_edge!(g,69,79)
add_edge!(g,69,80)
add_edge!(g,70,71)
add_edge!(g,70,80)
add_edge!(g,70,81)
add_edge!(g,71,72)
add_edge!(g,71,81)
add_edge!(g,71,82)
add_edge!(g,72,73)
add_edge!(g,72,82)
add_edge!(g,72,83)
add_edge!(g,73,74)
add_edge!(g,73,83)
add_edge!(g,73,84)
add_edge!(g,74,75)
add_edge!(g,74,84)
add_edge!(g,74,85)
add_edge!(g,75,76)
add_edge!(g,75,85)
add_edge!(g,75,86)
add_edge!(g,76,77)
add_edge!(g,76,86)
add_edge!(g,76,87)
add_edge!(g,77,78)
add_edge!(g,77,87)
add_edge!(g,77,88)
add_edge!(g,78,88)
add_edge!(g,78,89)
add_edge!(g,79,80)
add_edge!(g,79,90)
add_edge!(g,80,81)
add_edge!(g,80,91)
add_edge!(g,81,82)
add_edge!(g,81,92)
add_edge!(g,82,83)
add_edge!(g,82,93)
add_edge!(g,83,84)
add_edge!(g,83,94)
add_edge!(g,84,85)
add_edge!(g,84,95)
add_edge!(g,85,86)
add_edge!(g,85,96)
add_edge!(g,86,87)
add_edge!(g,96,97)
add_edge!(g,87,88)
add_edge!(g,87,98)
add_edge!(g,88,89)
add_edge!(g,88,98)
add_edge!(g,88,99)
add_edge!(g,89,99)
add_edge!(g,90,91)
add_edge!(g,91,92)
add_edge!(g,92,93)
add_edge!(g,93,94)
add_edge!(g,94,95)
add_edge!(g,95,96)
add_edge!(g,96,97)
add_edge!(g,97,98)
add_edge!(g,98,99)
    
    
    
    
   return g 
end

make_iowa_graph (generic function with 1 method)

In [ ]:
@time print(iowa_enumerator([1,2],50   ))